In [1]:
import os
os.chdir('..')

In [2]:
## it’s mine, my own, 
from modules.get_data import get_x_y
from jh_utils.data.pandas.preprocessing import make_dummies
from modules.evaluation import fit_classify, print_2_reports, plot_false_negative_rate

In [3]:
import pandas as pd, numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
## posfit
from sklearn.metrics import roc_auc_score, precision_recall_curve, det_curve
from sklearn.metrics import RocCurveDisplay, PrecisionRecallDisplay
from sklearn.metrics import classification_report,confusion_matrix

In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras import Sequential

2023-07-17 18:44:16.652653: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-17 18:44:17.239532: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
x,y = get_x_y()
x_train,x_test,y_train,y_test = train_test_split(x, y, test_size=0.3, stratify=y, random_state=42)
scaler = MinMaxScaler()
#scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

/home/joaoholz/Documents/ufes/trabalho_final/modules/get_data.py:6: DtypeWarning: Columns (3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv("asteroides/asteoides.csv").iloc[:, first_columns:]


In [6]:
def declare_model(n_layers=[16,8], loss = "binary_crossentropy", optimizer = "adam", last_layer_activation = "sigmoid"):
    tf_model = Sequential()
    tf_model.add(Dense(10, activation='relu', input_shape=(38,)))
    for i in n_layers[1:]:
        tf_model.add(Dense(i, activation='relu'))
    tf_model.add(Dense(1, activation=last_layer_activation))
    tf_model.compile(loss=loss, optimizer=optimizer, metrics = [tf.keras.metrics.Precision(),'accuracy'])
    return tf_model

In [7]:
tf.random.set_seed(2023)
tf_model = declare_model([16,8])

2023-07-17 18:44:25.741019: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-17 18:44:25.768008: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-17 18:44:25.768266: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [8]:
tf_model.fit(x_train,y_train, epochs=50, batch_size=64, verbose=1)

Epoch 1/50


2023-07-17 18:44:26.953810: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7fa951e77d50 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-07-17 18:44:26.953836: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce RTX 2060, Compute Capability 7.5
2023-07-17 18:44:26.959276: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-07-17 18:44:26.975171: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8902
2023-07-17 18:44:27.049496: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-07-17 18:44:27.118184: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


10198/10198 [==============================] - 29s 3ms/step - loss: 0.0171 - precision: 0.0035 - accuracy: 0.9952
Epoch 2/50
10198/10198 [==============================] - 25s 2ms/step - loss: 0.0058 - precision: 0.0000e+00 - accuracy: 0.9978
Epoch 3/50
10198/10198 [==============================] - 25s 2ms/step - loss: 0.0053 - precision: 0.0000e+00 - accuracy: 0.9978
Epoch 4/50
10198/10198 [==============================] - 25s 2ms/step - loss: 0.0048 - precision: 0.0000e+00 - accuracy: 0.9978
Epoch 5/50
10198/10198 [==============================] - 24s 2ms/step - loss: 0.0046 - precision: 0.0000e+00 - accuracy: 0.9978
Epoch 6/50
10198/10198 [==============================] - 25s 2ms/step - loss: 0.0044 - precision: 0.4524 - accuracy: 0.9978
Epoch 7/50
10198/10198 [==============================] - 25s 2ms/step - loss: 0.0043 - precision: 0.5010 - accuracy: 0.9978
Epoch 8/50
10198/10198 [==============================] - 25s 2ms/step - loss: 0.0042 - precision: 0.5299 - accuracy: 0.

In [9]:
y_prob = tf_model.predict(x_test)

8741/8741 [==============================] - 9s 1ms/step


In [10]:
y_pred = list(map(lambda x: x>0.5, y_prob))
print_2_reports(y_test,y_pred)

y_pred = list(map(lambda x: x>0.25, y_prob))
print_2_reports(y_test,y_pred)

y_pred = list(map(lambda x: x>0.05, y_prob))
print_2_reports(y_test,y_pred)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    279081
           1       0.95      0.63      0.76       620

    accuracy                           1.00    279701
   macro avg       0.98      0.81      0.88    279701
weighted avg       1.00      1.00      1.00    279701

        N    Y
N  279062   19
Y     232  388
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    279081
           1       0.87      0.80      0.83       620

    accuracy                           1.00    279701
   macro avg       0.93      0.90      0.92    279701
weighted avg       1.00      1.00      1.00    279701

        N    Y
N  279007   74
Y     125  495
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    279081
           1       0.70      0.95      0.80       620

    accuracy                           1.00    279701
   macro avg       0.85      0.97     

----

## 5 layer model

---

In [11]:
tf_model.fit(x_train,y_train, epochs=25, batch_size=128, verbose=1)

Epoch 1/25
5099/5099 [==============================] - 12s 2ms/step - loss: 0.0028 - precision: 0.7219 - accuracy: 0.9987
Epoch 2/25
5099/5099 [==============================] - 13s 2ms/step - loss: 0.0029 - precision: 0.7076 - accuracy: 0.9987
Epoch 3/25
5099/5099 [==============================] - 12s 2ms/step - loss: 0.0028 - precision: 0.7205 - accuracy: 0.9987
Epoch 4/25
5099/5099 [==============================] - 12s 2ms/step - loss: 0.0028 - precision: 0.7222 - accuracy: 0.9987
Epoch 5/25
5099/5099 [==============================] - 12s 2ms/step - loss: 0.0028 - precision: 0.7264 - accuracy: 0.9988
Epoch 6/25
5099/5099 [==============================] - 12s 2ms/step - loss: 0.0027 - precision: 0.7435 - accuracy: 0.9988
Epoch 7/25
5099/5099 [==============================] - 13s 3ms/step - loss: 0.0027 - precision: 0.7331 - accuracy: 0.9988
Epoch 8/25
5099/5099 [==============================] - 14s 3ms/step - loss: 0.0026 - precision: 0.7408 - accuracy: 0.9988
Epoch 9/25
5099/

In [12]:
y_prob = tf_model.predict(x_test)

8741/8741 [==============================] - 8s 871us/step


In [15]:
y_pred = list(map(lambda x: x>0.5, y_prob))
print_2_reports(y_test,y_pred)

y_pred = list(map(lambda x: x>0.25, y_prob))
print_2_reports(y_test,y_pred)

y_pred = list(map(lambda x: x>0.05, y_prob))
print_2_reports(y_test,y_pred)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    279081
           1       0.76      0.86      0.81       620

    accuracy                           1.00    279701
   macro avg       0.88      0.93      0.90    279701
weighted avg       1.00      1.00      1.00    279701

        N    Y
N  278916  165
Y      89  531
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    279081
           1       0.64      0.95      0.77       620

    accuracy                           1.00    279701
   macro avg       0.82      0.97      0.88    279701
weighted avg       1.00      1.00      1.00    279701

        N    Y
N  278755  326
Y      31  589
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    279081
           1       0.47      1.00      0.64       620

    accuracy                           1.00    279701
   macro avg       0.73      1.00     

Em especial para chegar no modelo de rede escolhido passamos pelas seguintes iterações:

| loss | opt  | ll act  | metric    | dropout    | layers   | data      | epoch | batch    |
| ---- | ---- | ------- | --------- | ---------- | -------- | --------- | ----- | -------- |
| BC   | rms  | sigmoid | 143/620   | True       | 2 layers | no sigmas | ~30   | no notes |
| mse  | adam | sigmoid | 134/620   | True       | 2 layers | no sigmas | ~30   | no notes |
| BC   | adam | sigmoid | 49/620    | True       | 2 layers | no sigmas | ~30   | no notes |
| BC   | adam | sigmoid | 58-12/620 | True       | 2 layers | no sigmas | ~30   | no notes |
| BC   | adam | sigmoid | 78-54/620 | no dropout | 1 layer  | no sigmas | ~30   | no notes |
| BC   | adam | sigmoid | 26-5/620  | no dropout | 2 layers | no sigmas | ~30   | no notes |
| BC   | adam | sigmoid | 48-18/620 | no dropout | 3 layers | no sigmas | ~30   | no notes |
| BC   | adam | sigmoid | 40-15/620 | no dropout | 2 layers | full data | ~30   | no notes |
| BC   | adam | sigmoid | 18-9/620  | no dropout | 3 layers | full data | ~30   | no notes |
| BC   | adam | tanh    | 120/620   | no dropout | 3 layers | full data | ~30   | no notes |
| BC   | adam | sigmoid | 5-1/620   | no dropout | 3 layers | full data | 20    | 128      |
| BC   | adam | sigmoid | 5-1/620   | no dropout | 3 layers | full data | 40    | 128      |
| BC   | adam | sigmoid | 5-1/620   | no dropout | 3 layers | full data | 40    | 64       |

-----

In [41]:
tf.random.set_seed(2023)
tf_model = declare_model([16,16,10])

In [42]:
tf_model.n_params()

AttributeError: 'Sequential' object has no attribute 'n_params'

In [16]:
tf_model.fit(x_train,y_train, epochs=100, batch_size=64, verbose=1)

Epoch 1/100
10198/10198 [==============================] - 27s 3ms/step - loss: 0.0085 - precision_1: 0.0000e+00 - accuracy: 0.9978
Epoch 2/100
10198/10198 [==============================] - 24s 2ms/step - loss: 0.0052 - precision_1: 0.0000e+00 - accuracy: 0.9978
Epoch 3/100
10198/10198 [==============================] - 28s 3ms/step - loss: 0.0047 - precision_1: 0.6591 - accuracy: 0.9978
Epoch 4/100
10198/10198 [==============================] - 27s 3ms/step - loss: 0.0045 - precision_1: 0.5290 - accuracy: 0.9978
Epoch 5/100
10198/10198 [==============================] - 27s 3ms/step - loss: 0.0044 - precision_1: 0.5446 - accuracy: 0.9978
Epoch 6/100
10198/10198 [==============================] - 28s 3ms/step - loss: 0.0042 - precision_1: 0.5552 - accuracy: 0.9979
Epoch 7/100
10198/10198 [==============================] - 26s 3ms/step - loss: 0.0041 - precision_1: 0.5785 - accuracy: 0.9980
Epoch 8/100
10198/10198 [==============================] - 25s 2ms/step - loss: 0.0040 - precisi

In [ ]:
y_prob = tf_model.predict(x_test)

In [ ]:
plot_false_negative_rate(y_test,y_prob)

In [23]:
y_pred = list(map(lambda x: x>0.5, y_prob))
print_2_reports(y_test,y_pred)

y_pred = list(map(lambda x: x>0.25, y_prob))
print_2_reports(y_test,y_pred)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    279081
           1       0.92      0.95      0.93       620

    accuracy                           1.00    279701
   macro avg       0.96      0.98      0.97    279701
weighted avg       1.00      1.00      1.00    279701

        N    Y
N  279028   53
Y      30  590
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    279081
           1       0.78      0.99      0.87       620

    accuracy                           1.00    279701
   macro avg       0.89      1.00      0.94    279701
weighted avg       1.00      1.00      1.00    279701

        N    Y
N  278907  174
Y       5  615


In [22]:
y_pred = list(map(lambda x: x>0.15, y_prob))
print_2_reports(y_test,y_pred)

y_pred = list(map(lambda x: x>0.05, y_prob))
print_2_reports(y_test,y_pred)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    279081
           1       0.72      1.00      0.84       620

    accuracy                           1.00    279701
   macro avg       0.86      1.00      0.92    279701
weighted avg       1.00      1.00      1.00    279701

        N    Y
N  278838  243
Y       1  619
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    279081
           1       0.62      1.00      0.77       620

    accuracy                           1.00    279701
   macro avg       0.81      1.00      0.88    279701
weighted avg       1.00      1.00      1.00    279701

        N    Y
N  278704  377
Y       0  620



279028   53
    30  590
278838  243
     1  619
278704  377
     0  620